In [91]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from sklearn.model_selection import train_test_split

In [89]:
X_train_val = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/X_train_val.csv').set_index('fire_id')
y_train_val = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/y_train_val.csv').set_index('fire_id')

In [92]:
explanatory = X_train_val
target =  y_train_val
X_train, X_val, y_train, y_val = train_test_split(explanatory,
                                                target,
                                                test_size=0.15,
                                                shuffle=True,
                                                random_state=1,
                                                stratify=target)

---

In [22]:
df = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/table_data_input.csv')
df.drop(columns=df.columns[0], inplace=True)
df.drop_duplicates(inplace=True)

/tmp/ipykernel_4099218/2703120438.py:1: DtypeWarning: Columns (331) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/table_data_input.csv')


In [31]:
df.iloc[:, 330]

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
10880    0.0
10881    0.0
10882    0.0
10883    0.0
10884    0.0
Name: VEGETAZIONE_INTERVENTO_Ceduazione a ceduo semplice, Length: 10855, dtype: object

In [38]:
df[df.iloc[:, 330].str.contains('POINT').fillna(False)]

/tmp/ipykernel_4099218/4216348789.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[df.iloc[:, 330].str.contains('POINT').fillna(False)]


,analfabeti,alfabeti privi di titolo di studio,licenza di scuola elementare,licenza di scuola media inferiore o di avviamento professionale,diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS,diploma di tecnico superiore ITS o titolo di studio terziario di primo livello,titolo di studio terziario di secondo livello e dottorato di ricerca,titolo di studio terziario di secondo livello,dottorato di ricerca/diploma accademico di formazione alla ricerca,totale_istruzione,...,durata_calma_12,umedia_12,umin_12,umax_12,rtot_12,hdd_base18_12,hdd_base20_12,cdd_base18_12,YYYY,ignition_point (epsg=3857)
981,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
df.loc[981, 'ignition_point (epsg=3857)'] = df.loc[981, :].iloc[330]

In [44]:
df.loc[981, df.columns[330]] = 0.0

In [46]:
df.loc[:, df.columns[330]] = df.loc[:, df.columns[330]].astype('float64')

In [48]:
df.to_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/table_data_input.csv', index=False)

---

In [50]:
df = pd.read_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/table_data_input.csv')
gdf = gpd.read_file('/nfs/home/genovese/thesis-wildfire-genovese/database/cache_resampled/dataset.geojson')

In [65]:
df['ignition_point (epsg=3857)'] = df['ignition_point (epsg=3857)'].apply(wkt.loads)

In [72]:
print(df.iloc[0, -1])
print(gdf.iloc[0, -1])

POINT (887672.0754000134 5659250.455444928)
POINT (841872.0754000134 5532450.455444928)


In [82]:
gdf.YYYY = gdf.YYYY.astype('float64')

In [84]:
merged = pd.merge(gdf, df.rename(columns={'ignition_point (epsg=3857)': 'geometry'}), on=['geometry', 'YYYY'], how='inner')

In [85]:
merged.set_index('fire_id', inplace=True)

In [87]:
merged.to_csv('/nfs/home/genovese/thesis-wildfire-genovese/database/model_input_resampled/table_data_input_with_target.csv')

---